# Code for Traveling Salesman Problem

### Model by Miller, Tucker and Zemlin

\begin{equation}
\begin{split}
\min~ & \sum_{i=1}^n\sum_{j=1}^nc_{ij}x_{ij} \\
s.t. ~& \sum_{i=1}^nx_{ij} =1, ~ j =1,...,n\\
& \sum_{j=1}^nx_{ij} =1, ~ i =1,...,n\\
& u_i + 1 - u_j \leq M(1 - x_{ij}), ~ i\neq j, 1\leq i\leq n, 2\leq j \leq n \\ 
& x_{ij}\in\{0,1\}, u_i\geq 0, i =1,...,n.
\end{split}
\end{equation}

In [1]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

#traveling cost from node i to node j
cost = np.array([[1000, 4, 2, 1000, 1000, 1000],
                 [0, 1000, 6, 1000, 1, 7], 
                 [0, 6, 1000, 2, 1, 1000],
                 [0, 1000, 2, 1000, 1, 3],
                 [0, 1, 1, 1, 1000, 1], 
                 [0, 7, 1000, 3, 1, 1000]])

N = cost.shape[0]

#the big M
M = 10000

In [2]:
#########Model Set-up############


tsp = Model("traveling_salesman")

# Creat variables
x = tsp.addVars(N, N, vtype=GRB.BINARY, name = "x")

u = tsp.addVars(N, name = "u")

# Set objective
tsp.setObjective( quicksum(cost[i,j]*x[i,j] for i in range(N) for j in range(N)), GRB.MINIMIZE)

# Assignment constraints:
tsp.addConstrs(( quicksum(x[i,j] for j in range(N)) == 1 for i in range(N) ))
 
tsp.addConstrs(( quicksum(x[i,j] for i in range(N)) == 1 for j in range(N) ))

# Subtour-breaking constraints:
tsp.addConstrs(( u[i] + 1 - u[j] <= M*(1 - x[i,j])  for i in range(N) for j in range(1,N) ))


# Solving the model
tsp.optimize()


Academic license - for non-commercial use only
Optimize a model with 42 rows, 42 columns and 152 nonzeros
Variable types: 6 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 2010.0000000
Presolve removed 5 rows and 6 columns
Presolve time: 0.11s
Presolved: 37 rows, 36 columns, 152 nonzeros
Variable types: 5 continuous, 31 integer (31 binary)

Root relaxation: objective 9.000000e+00, 23 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.00000    0    4 2010.00000    9.00000   100%     -    0s
H    0     0                       9.0000000    9.00000  0.00%     -    0s

Explored 1 nodes (23 simplex iterations) in 0.44 seconds
Thread count was 4 (of 4 available processors

In [3]:
#  Print optimal x for x nonzero and optimal value
s_edge = []
for v in x:    
    if x[v].x > 0.001:
        print(x[v].VarName, x[v].x)
        #add both of the indicies by 1
        edge = np.add(v, (1,1))
        #append the edge to the resulting list of edges
        s_edge.append(edge)


print('Obj:', tsp.objVal)
print(s_edge)
for v in u: 
    print(u[v].VarName, u[v].x)

x[0,2] 1.0
x[1,0] 1.0
x[2,3] 1.0
x[3,5] 1.0
x[4,1] 1.0
x[5,4] 1.0
Obj: 9.0
[array([1, 3]), array([2, 1]), array([3, 4]), array([4, 6]), array([5, 2]), array([6, 5])]
u[0] 0.0
u[1] 5.0
u[2] 1.0
u[3] 2.0
u[4] 4.0
u[5] 3.0


In [44]:
#  Obtain the permutation as a representation of the tour

permu = np.ones(N)
predecessor = 1
for i in range(N):
    for s in s_edge:
        if s[0] == predecessor:
            permu[i] = s[0]
            predecessor = s[1]
            break    
    
print(permu)

[1. 3. 4. 6. 5. 2.]


### Precedence constraint:
\begin{equation}
u_i\leq u_j, ~ \forall (i,j)\in P.
\end{equation}

In [50]:
# data for the precedent pair

Precedent_Pair = tuplelist([(0,2), (1,3), (4,5)])


tsp.addConstrs( (u[i] <= u[j] for (i,j) in Precedent_Pair)  )


# Solving the new model
tsp.optimize()






Optimize a model with 45 rows, 42 columns and 158 nonzeros
Variable types: 6 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start did not produce a new incumbent solution

Found heuristic solution: objective 1016.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.03s
Presolved: 39 rows, 36 columns, 143 nonzeros
Variable types: 5 continuous, 31 integer (31 binary)

Root relaxation: objective 9.000200e+00, 22 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.00020    0    6 1016.00000    9.00020  99.1%     -    0s
H    0     0                    1007.0000000    9.00020  99.1%     -    0s
H    0     0                      14.0000000    9.00020  35.7%     -    0s
H    0     0          

In [51]:
#  The list of edges traversed
s_edge = []
for v in x:    
    if x[v].x > 0.001:
        edge = np.add(v, (1,1))
        s_edge.append(edge)
        
#  Obtain the permutation as a representation of the tour
permu = np.ones(N)
predecessor = 1
for i in range(N):
    for s in s_edge:
        if s[0] == predecessor:
            permu[i] = s[0]
            predecessor = s[1]
            break    
    
print(permu)

[1. 2. 5. 3. 4. 6.]
